# Load Libraries

In [1]:
import os
import json
import math
import random

import pandas as pd

from models import Model_Multiperiod
from classes import  Satellite,Customer,Vehicle
from drawingmap import DrawingMap

# Functions

## Load Customers

In [29]:
def load_customers(DEBUG = False) -> list[Customer]:
    customers = []
    df = pd.read_csv('../others/Levantamiento de Información/customers_20210119.csv')
    # filtrado
    df = df[(df.category < 16000) & (df.cajas > 0)].reset_index().copy()
    df['isSmall'] = df.apply(lambda x: True if x['cajas']<=5 else False, axis=1)
    #
    for i in range(len(df)):
        new_customer = Customer(id=str(df.id_cliente[i])
                                , lon=df.loc[i, 'lon']
                                , lat=df.loc[i, 'lat']
                                , demand=[df.loc[i, 'cajas'] for j in range(10)]
                                , category=df.loc[i, 'category']
                                , isLow=df.loc[i, 'isSmall']
                                , fee_min_dc=0
                                , fee_min_satellite=0
                                )
        customers.append(new_customer)
    if DEBUG:
        print("-"*50)
        print("Count of customers: ", len(customers))
        print("First customer:")
        print(json.dumps(customers[0].__dict__,indent=2,default=str))
    return customers, df

## Load Satelites

In [30]:
def load_satellites(DEBUG = False) ->list[Satellite]:
    satellites = []
    df = pd.read_csv('../others/Levantamiento de Información/Informacion CD a Satelites.csv')
    df.fillna(0, inplace=True)
    #
    df['costFixed'] = [10000, 12000, 14000, 9000, 8000, 10000, 12000, 15000, 9000]
    df['costPerVehicle'] = [120, 140, 100, 90, 100, 120, 110, 100, 150]
    #
    for i in range(len(df)):
        newSatellite = Satellite(id=str(df.nombre[i])
                               , lon=df.longitud[i]
                               , lat=df.latitud[i]
                               , distanceFromDC=df.loc[i, 'distance.value']/1000
                               , durationFromDC=df.loc[i, 'duration.value']/3600
                               , durationInTrafficFromDC=df.loc[i, 'duration_in_traffic.value']/3600
                               , costFixed=df.loc[i, 'costFixed']
                               , costPerVehicle=df.loc[i, 'costPerVehicle']
                               )
        satellites.append(newSatellite)
    if DEBUG:
        print("-"*50)
        print("Count of SATELLITES: ", len(satellites))
        print("First Satellite:")
        print(json.dumps(satellites[0].__dict__,indent=2,default=str))
    return satellites, df

## Load Vehicle

In [31]:
small_vehicle = Vehicle(id='small'
                        , capacity=20
                        , cost=10)
large_vehicle = Vehicle(id='large'
                        , capacity=120
                        , cost=50)

## Load Matrixes: Distance and Durations with and without traffic

In [32]:
def distance(s: Satellite, c: Customer) -> float:
     return int(math.sqrt(math.pow(s.lat - c.lat, 2) + math.pow(s.lon - c.lon, 2))*1000)

In [33]:
def calculate_distance(satellites: list[Satellite], customers: list[Customer], DEBUG = False) -> dict[(str,str), float]:
    distance_matrix = dict(
        [((s.id, c.id), distance(s,c)) for s in satellites for c in customers]
    )
    if DEBUG:
        print("-"*50)
        print("Count - distance: ", len(distance_matrix))
    return distance_matrix

# Drawing Map

In [34]:
s, df_s = load_satellites(True)

--------------------------------------------------
Count of SATELLITES:  9
First Satellite:
{
  "id": "Abaroa",
  "lon": -68.128476,
  "lat": -16.512483,
  "geographyLocation": [
    -68.128476,
    -16.512483
  ],
  "distanceFromDC": 11.117,
  "durationFromDC": 0.5772222222222222,
  "durationInTrafficFromDC": 0.6044444444444445,
  "costFixed": "10000",
  "costPerVehicle": "120"
}


In [35]:
df_s = df_s[["latitud",'longitud']]
df_s.columns = ['lat','lon']
df_s.head()

,lat,lon
0,-16.512483,-68.128476
1,-16.527796,-68.129577
2,-16.538659,-68.060496
3,-16.483875,-68.147618
4,-16.570317,-68.085339


In [36]:
c, df_c = load_customers(True)

--------------------------------------------------
Count of customers:  1468
First customer:
{
  "id": "44821",
  "lon": -68.14302642,
  "lat": -16.50004048,
  "geographyLocation": [
    -68.14302642,
    -16.50004048
  ],
  "demand": [
    3.0,
    3.0,
    3.0,
    3.0,
    3.0,
    3.0,
    3.0,
    3.0,
    3.0,
    3.0
  ],
  "category": "13024",
  "isLow": "True",
  "fee_min_satellite": 0,
  "fee_min_dc": 0
}


In [12]:
display(large_vehicle.__dict__)
display(small_vehicle.__dict__)

{'id': 'large', 'capacity': 120, 'cost': 50}

{'id': 'small', 'capacity': 20, 'cost': 10}

In [14]:
df_c_small = df_c[df_c.isSmall == True].reset_index()
df_c_small.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1220 entries, 0 to 1219
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   level_0     1220 non-null   int64  
 1   index       1220 non-null   int64  
 2   id_cliente  1220 non-null   int64  
 3   lon         1220 non-null   float64
 4   lat         1220 non-null   float64
 5   cajas       1220 non-null   float64
 6   category    1220 non-null   int64  
 7   isSmall     1220 non-null   bool   
dtypes: bool(1), float64(3), int64(4)
memory usage: 68.0 KB


In [15]:
df_c_large = df_c[df_c.isSmall == False].reset_index()
df_c_large.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   level_0     248 non-null    int64  
 1   index       248 non-null    int64  
 2   id_cliente  248 non-null    int64  
 3   lon         248 non-null    float64
 4   lat         248 non-null    float64
 5   cajas       248 non-null    float64
 6   category    248 non-null    int64  
 7   isSmall     248 non-null    bool   
dtypes: bool(1), float64(3), int64(4)
memory usage: 13.9 KB


In [16]:
distance_matrix = calculate_distance(s,c)

## La Paz with Customers

In [17]:
location_la_paz = [-16.501457, -68.149887]
drawer = DrawingMap(location_la_paz)
#drawer.addCustomers(c)
drawer.addNodes(df_c_small, color="blue", radius=1)
drawer.addNodes(df_c_large, color="yellow", radius=1)

drawer.addNodes(df_s,color='red')
map = drawer.viewMap()
map

## Params for Model

In [19]:
params = {
    'large_vehicle': large_vehicle,
    'small_vehicle': small_vehicle
}
periods = 4

# Model


In [21]:
model_deterministic = Model_Multiperiod(periods=periods)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-13


In [ ]:
model_deterministic.build(

)

In [ ]:
print(model_deterministic.optimizeModel())

In [ ]:
model_deterministic.model.getObjective().getValue()